In [1]:
from selenium import webdriver
import requests
import time
import pandas as pd
from IPython.display import clear_output

In [2]:
# webdriver 실행
driver = webdriver.Chrome('chromedriver')

In [3]:
# 다음 영화 사이트 접속
site = 'https://movie.daum.net/premovie/theater'
time.sleep(1)
driver.get(site)

In [4]:
# 스크롤:
for idx in range(10):
    time.sleep(1)
    script = 'window.scrollTo(0, document.body.scrollHeight)'
    driver.execute_script(script)

In [5]:
# 영화 전체 목록 가져오기
a1 = driver.find_elements_by_css_selector('#mainContent > div > div.box_movie > ol')
a1

[<selenium.webdriver.remote.webelement.WebElement (session="26c74ee5f1407428b23ba1d7b9e34b59", element="8ff707c4-50cd-495f-82f8-86ccc3a32aa1")>]

In [6]:
# 영화 전체 목록에서 li tag 가져오기 
li_list = a1[0].find_elements_by_css_selector('li')
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="26c74ee5f1407428b23ba1d7b9e34b59", element="9ab43d24-aa44-41a3-bff8-8866e72fa028")>,
 <selenium.webdriver.remote.webelement.WebElement (session="26c74ee5f1407428b23ba1d7b9e34b59", element="5810918c-5d5a-4ad3-b0fe-4cc4c699329b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="26c74ee5f1407428b23ba1d7b9e34b59", element="857b75d3-1b93-491d-9212-e884f8dff356")>,
 <selenium.webdriver.remote.webelement.WebElement (session="26c74ee5f1407428b23ba1d7b9e34b59", element="44797556-9a45-4fe8-b0b8-2f58461af2f9")>,
 <selenium.webdriver.remote.webelement.WebElement (session="26c74ee5f1407428b23ba1d7b9e34b59", element="c96888ad-67f4-465e-aca4-5f0ea9194641")>,
 <selenium.webdriver.remote.webelement.WebElement (session="26c74ee5f1407428b23ba1d7b9e34b59", element="7b8d0943-f09d-44ba-8b13-3de36fcff04e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="26c74ee5f1407428b23ba1d7b9e34b59", element="198f3c7d-9ea8-459a-b498-d8

In [7]:
# 데이터를 담을 딕셔너리 
data_dict = {
    '제목' : [],
    '예매율' : [],
    '평점' : [],
    '개봉일' : []
}

# 영화의 수 만큼 반복
for movie_tag in li_list:
    # 영화 제목 가져오기
    title_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > strong > a')
    title = title_tag.text.strip()
    
    # 평점
    rating_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(1) > span')
    rating = rating_tag.text.strip()
    
    # 예매율
    ticket_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_append > span:nth-child(2) > span')
    ticket = ticket_tag.text.strip()
    
    # 개봉일
    open_date_tag = movie_tag.find_element_by_css_selector('div > div.thumb_cont > span.txt_info > span')
    open_date = open_date_tag.text.strip()
    
    # 영화 포스터
    poster_tag = movie_tag.find_element_by_css_selector('div > div.thumb_item > div.poster_movie > img')
    # src 속성의 값 가져오기
    src_attr = poster_tag.get_attribute('src')
#     print(src_attr)
    
    # 영화의 제목을 파일명으로 사용할 것이기 때문에
    # os에서 거부하는 파일명을 정제한다
    char_list = ['\\', '/', ':', '*', '?', '"', '<', '>', '|']
    
    file_name = title

    for c1 in char_list:
        file_name = file_name.replace(c1, ' ')
    
    # 이미지 데이터 내려받기
    img_res = requests.get(src_attr)
    
    # 저장! 
    with open(f'poster/{file_name}.jpg', 'wb') as fp:
        fp.write(img_res.content)

    
#     print(title)
#     print(rating)
#     print(ticket[:-1])
#     print(open_date)
#     print('------------------------')

    # 데이터 담기
    data_dict['제목'].append(title)
    data_dict['예매율'].append(ticket[:-1])
    data_dict['평점'].append(rating)
    data_dict['개봉일'].append(open_date)
    
# DataFrame 생성
df1 = pd.DataFrame(data_dict)

# 저장
df1.to_csv('daum_movie.csv', encoding='utf-8-sig', index=False)
print('저장완료')

저장완료
